In [1]:
EXPORT_PATH = '../data/processed/'
DATA_PATH = '../data/raw/'

target_col= ['DevType']
influnce_col = ['LanguageWorkedWith',
                  'DatabaseWorkedWith',
                  'PlatformWorkedWith',
                  'WebframeWorkedWith',
                  'MiscTechWorkedWith',
                  'NEWCollabToolsWorkedWith']


# import lib

In [2]:
import pandas as pd 
import numpy as np
import logging
import pickle
import os

import plotly 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff

from sklearn.manifold import TSNE
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.cluster import AgglomerativeClustering

from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram



## export my data frame

In [3]:
raw_df=pd.read_pickle(EXPORT_PATH+'1_preprocessed.pkl')


EOFError: Ran out of input

## one hot encoding

In [ ]:
df=raw_df.copy()
encoded_dfs = {}
for col in target_col + influnce_col:
    binarizer = MultiLabelBinarizer()
    encoded_df = pd.DataFrame(binarizer.fit_transform(df[col]),
                               columns=binarizer.classes_,
                               index=df[col].index)
    encoded_dfs[col] = encoded_df

In [ ]:
df= pd.concat(encoded_dfs,axis=1)


In [ ]:
skill = df.copy().drop(columns=['DevType']).sum().sort_values(ascending=False).reset_index()
skill.columns = ['group','skill','count']
px.treemap(skill,path=['group','skill'],values='count',title='skills',width=1000,height=1000)
#skill

In [ ]:
jobs_freq = df['DevType'].sum().reset_index()
jobs_freq.columns = ['job','count']

px.treemap(jobs_freq,path=['job'],values='count',title='jobs',width=1000,height=1000)

In [ ]:
sorted_roles = df['LanguageWorkedWith'].sum().sort_values(ascending=False).index.tolist()

sorted_roles

['JavaScript',
 'HTML/CSS',
 'SQL',
 'Python',
 'Java',
 'Bash/Shell/PowerShell',
 'C#',
 'PHP',
 'TypeScript',
 'C++',
 'C',
 'Go',
 'Kotlin',
 'Ruby',
 'Assembly',
 'VBA',
 'Swift',
 'R',
 'Rust',
 'Objective-C',
 'Dart',
 'Scala',
 'Perl',
 'Haskell',
 'Julia']

In [ ]:
sorted_roles = df['DevType'].sum().sort_values(ascending=False).index.tolist()
sorted_roles

['Developer, back-end',
 'Developer, full-stack',
 'Developer, front-end',
 'Developer, desktop or enterprise applications',
 'Developer, mobile',
 'DevOps specialist',
 'Database administrator',
 'Designer',
 'System administrator',
 'Developer, embedded applications or devices',
 'Data or business analyst',
 'Data scientist or machine learning specialist',
 'Developer, QA or test',
 'Engineer, data',
 'Academic researcher',
 'Educator',
 'Developer, game or graphics',
 'Engineering manager',
 'Product manager',
 'Scientist',
 'Engineer, site reliability',
 'Senior executive/VP',
 'Marketing or sales professional']

In [ ]:
skills =[]
for role in sorted_roles:
    role_mask = (df[('DevType', role)] == 1)
    skills_role = pd.concat({tech_col: df.loc[role_mask,tech_col].mean() * 100
                             for tech_col in influnce_col})
    
    
    skills.append(skills_role)
     


skills = pd.concat(skills, axis=1)
skills.columns = sorted_roles
skills = skills.reset_index(level=0,drop=True)
skills=skills.T
skills

,Assembly,Bash/Shell/PowerShell,C,C#,C++,Dart,Go,HTML/CSS,Haskell,Java,...,Facebook Workplace,Github,Gitlab,"Google Suite (Docs, Meet, etc)",Jira,Microsoft Azure,Microsoft Teams,Slack,Stack Overflow for Teams,Trello
"Developer, back-end",4.745115,35.713236,17.173498,34.023799,19.244895,3.404584,11.146614,61.389011,2.001616,39.848685,...,2.754517,74.294844,36.785662,38.739533,49.911855,16.068018,25.672102,51.722492,4.976495,28.356839
"Developer, full-stack",3.845161,33.500461,14.422120,36.376037,16.250691,3.911521,9.209217,73.784332,1.692166,36.781567,...,2.573272,75.395392,35.520737,39.347465,48.247005,17.518894,26.654378,52.733641,4.925346,30.348387
"Developer, front-end",3.973546,29.334281,14.675339,33.542851,16.047223,4.394403,7.542632,78.312199,1.617840,33.728684,...,3.027984,75.726935,35.417578,39.713599,45.917140,15.954307,25.087451,51.470267,5.667906,31.351115
"Developer, desktop or enterprise applications",6.737950,36.210115,22.878479,51.332315,28.131365,3.699932,7.909029,61.311948,1.849966,40.733198,...,2.528853,69.730143,32.569586,32.569586,45.239308,21.987441,32.739308,41.352682,4.981331,24.151392
"Developer, mobile",5.357520,26.323560,19.521198,34.096182,22.516347,11.273993,7.403501,60.693946,1.497574,49.441046,...,3.680658,77.747311,36.753849,41.056739,45.074879,15.819447,22.463615,52.710399,6.580890,34.454756
DevOps specialist,5.327526,58.686547,17.574133,35.131513,18.261015,3.769476,19.919585,62.707321,2.479477,38.431898,...,3.451164,77.734964,43.910203,44.429553,59.038365,24.593734,32.819568,59.189144,4.657397,30.859440
Database administrator,6.850751,43.411395,19.189095,40.387976,20.447396,4.229290,10.013981,73.068857,2.009787,35.040196,...,3.005942,71.268787,33.974135,40.055925,37.172317,18.472562,26.476756,43.778399,6.308983,27.962251
Designer,7.028754,30.182297,22.533358,35.087390,24.130802,4.961473,6.558917,74.873144,1.935726,35.895508,...,3.476790,72.486375,31.685773,39.297125,34.636347,15.354257,24.018042,42.022176,8.024807,28.791581
System administrator,8.393743,54.940862,23.998474,31.896223,23.616940,4.101488,13.964136,69.496375,2.708890,35.806944,...,2.918733,73.636017,40.061045,43.132392,39.813048,16.081648,24.780618,46.375429,5.188859,28.462419
"Developer, embedded applications or devices",15.389474,43.978947,47.894737,38.336842,47.473684,5.663158,10.778947,54.105263,2.863158,39.831579,...,3.347368,72.968421,38.315789,37.052632,46.147368,16.673684,27.663158,42.589474,6.273684,24.315789


In [ ]:
fig = go.Figure(data=go.Heatmap(z=skills, x=skills.columns,y=skills.index, colorscale='magma', ygap=1))
fig.update_layout(width=1600, height=700)
fig.show()


In [ ]:
fig= ff.create_dendrogram(skills,orientation='left',labels=skills.index,color_threshold=0.5)
fig.update_layout(width=800,height=1000)
fig.show()


## Normalization

In [ ]:
from sklearn.preprocessing import StandardScaler
std_skills = StandardScaler().fit_transform(skills)
std_skills = pd.DataFrame(std_skills, columns=skills.columns, index=skills.index)
fig = go.Figure(data=go.Heatmap(z=std_skills, x=skills.columns,y=skills.index, colorscale='magma', ygap=1))
fig.update_layout(width=1600, height=700)
fig.show()


In [ ]:
fig= ff.create_dendrogram(std_skills,orientation='left',labels=std_skills.index,color_threshold=0.5)
fig.update_layout(width=800,height=1000)
fig.show()



## profile for roles

In [ ]:
role='Data scientist or machine learning specialist'
single_role_skills = pd.concat([skills.loc[role], std_skills.loc[role]], axis=1)
single_role_skills.columns = ['percentage', 'specificity']
single_role_skills = single_role_skills.sort_values(by='percentage',ascending=False)
single_role_skills

,percentage,specificity
Python,78.335005,2.614848
Github,78.059178,0.967076
Linux,63.540622,0.720565
SQL,58.450351,-0.180472
Slack,51.078235,0.167600
...,...,...
Unreal Engine,2.758275,-0.330042
Gatsby,2.683049,-0.732300
Puppet,2.432297,-0.425910
Couchbase,2.357071,-0.627059


In [ ]:
threshold = 25

single_role_skills = single_role_skills[single_role_skills['percentage'] > threshold]

fig = px.bar(df, 
             y=single_role_skills.index, 
             x=single_role_skills['percentage'], 
             color=single_role_skills['specificity'], 
             color_continuous_scale='orrd', 
             range_color=[std_skills.values.min(),std_skills.values.max()],
             orientation='h')

fig.update_layout(width=800, height=800, title=role)
fig.show()